# Bereitstellen auf einem Batchendpunkt

Stellen Sie sich vor, eine Klinik nimmt den ganzen Tag Patientenmessungen vor und speichert die Details für jeden Patienten in einer separaten Datei. Anschließend kann das Diabetesvorhersagemodell über Nacht verwendet werden, um alle Patientendaten des Tages als Batch zu verarbeiten und Vorhersagen zu generieren, die am nächsten Morgen bereitstehen. Damit kann die Klinik sich an Patienten wenden, für die ein Diabetesrisiko vorhergesagt wird. Mit Azure Machine Learning können Sie dies erreichen, indem Sie einen Batchendpunkt erstellen, den Sie in dieser Übung implementieren.

## Vorbereitung

Sie benötigen die neueste Version des Pakets **azureml-ai-ml**, um den Code in diesem Notebook auszuführen. Führen Sie die folgende Zelle aus, um zu überprüfen, ob das Paket installiert ist.

> **Hinweis**:
> Wenn das Paket **azure-ai-ml** nicht installiert ist, führen Sie `pip install azure-ai-ml` aus, um es zu installieren.

In [ ]:
## Herstellen einer Verbindung mit Ihrem Arbeitsbereich

Sie können nun eine Verbindung mit Ihrem Arbeitsbereich herstellen, nachdem Sie die erforderlichen SDK-Pakete installiert haben.

Um eine Verbindung mit einem Arbeitsbereich herzustellen, benötigen Sie Bezeichnerparameter: eine Abonnement-ID, einen Ressourcengruppennamen und einen Arbeitsbereichsnamen. Der Ressourcengruppenname und Arbeitsbereichsname sind bereits für Sie ausgefüllt. Sie müssen lediglich die Abonnement-ID angeben, um den Befehl zu vervollständigen.

Um die erforderlichen Parameter zu finden, klicken Sie rechts oben in Studio auf das Abonnement und den Namen des Arbeitsbereichs. Rechts wird ein Bereich geöffnet.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Kopieren Sie die Abonnement-ID, und ersetzen Sie **YOUR-SUBSCRIPTION-ID** durch den Wert, den Sie kopiert haben. </p>

## Registrieren des Modells

Bei Batchbereitstellungen können nur Modelle bereitgestellt werden, die im Arbeitsbereich registriert sind. Sie registrieren ein MLflow-Modell, das im lokalen Ordner `model` gespeichert ist. 

In [ ]:
## Erstellen eines Batchendpunkts

Ein Batchendpunkt ist ein HTTPS-Endpunkt, den Anwendungen aufrufen können, um einen Batchbewertungsauftrag auszulösen. Der Name eines Batchendpunkts muss innerhalb einer Azure-Region eindeutig sein. Mit der Funktion `datetime` generieren Sie einen eindeutigen Namen basierend auf dem aktuellen Datum und der aktuellen Uhrzeit. 

In [ ]:
Um einen Endpunkt mit der `BatchEndpoint`-Klasse zu erstellen, müssen Sie den Namen und optional eine Beschreibung angeben. Nachdem Sie einen Endpunkt erstellt haben, stellen Sie ein Modell darauf bereit.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> WICHTIG! Warten Sie, bis der Endpunkt erstellt wurde, ehe Sie fortfahren! In Studio sollte eine grüne Benachrichtigung eingeblendet werden. </p>

## Erstellen der Bereitstellung

Eine Einrichtung ist ein Satz von Ressourcen, die für das Hosting des Modells erforderlich sind, das die eigentliche Inferenz durchführt. Wir erstellen eine Bereitstellung für unseren Endpunkt mithilfe der `BatchDeployment`-Klasse. 

Da Sie ein MLflow-Modell bereitstellen, benötigen Sie weder ein Bewertungsskript noch müssen Sie die Umgebung definieren. Azure Machine Learning erstellt diese Ressourcen automatisch für Sie. Die Datei `MLmodel` im Ordner `model` wird verwendet, um die erwarteten Ein- und Ausgaben des Modells zu verstehen.

Sie stellen ein Modell mit den folgenden Parametern bereit:

- `name`: Name der Bereitstellung.
- `description`: optionale Beschreibung, um zu verdeutlichen, was die Bereitstellung darstellt.
- `endpoint_name`: Name des zuvor erstellten Endpunkts, auf dem das Modell bereitgestellt werden soll.
- `model`: Name des registrierten Modells.
- `compute`: Compute-Instanz, die beim Aufrufen des bereitgestellten Modells verwendet werden soll, um Vorhersagen zu generieren.
- `instance_count`: Anzahl der Serverknoten, die zum Generieren von Vorhersagen verwendet werden sollen
- `max_concurrency_per_instance`: maximale Anzahl paralleler Bewertungsskriptausführungen pro Serverknoten
- `mini_batch_size`: Anzahl der pro Bewertungsskriptausführung übergebenen Dateien
- `output_action`: Jede neue Vorhersage wird als neue Zeile an die Ausgabedatei angefügt.
- `output_file_name`: Datei, an die Vorhersagen angefügt werden.
- `retry_settings`: Einstellungen, wenn bei einem Minibatch ein Fehler auftritt.
- `logging_level`: die Ausführlichkeitsstufe des Protokolls. Zulässige Werte sind `warning`, `info` und `debug`. 

Wenn Sie die folgende Zelle ausführen, wird die Bereitstellung konfiguriert und erstellt.

In [ ]:
<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> WICHTIG! Warten Sie, bis die Bereitstellung abgeschlossen ist, ehe Sie fortfahren! In Studio sollte eine grüne Benachrichtigung eingeblendet werden. </p>

Sie können mehrere Modelle auf einem Endpunkt bereitstellen. Sie können die Standardbereitstellung festlegen, um anzugeben, welches Modell beim Aufrufen eines Batchendpunkts standardmäßig verwendet werden soll.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> WICHTIG! Warten Sie, bis die Standardbereitstellung festgelegt ist, ehe Sie fortfahren! In Studio sollte eine grüne Benachrichtigung eingeblendet werden. </p>

## Vorbereiten der Daten für Batchvorhersagen

Im Ordner `data` finden Sie CSV-Dateien mit Daten ohne Bezeichnung. Sie erstellen eine Datenressource, die auf die Dateien im Ordner `data` verweist, die als Eingabe für den Batchauftrag dienen.

In [ ]:
## Übermitteln des Auftrags

Sie haben ein Modell für einen Batchendpunkt bereitgestellt und verfügen über eine Datenressource ohne Bezeichnung. Nun können Sie den Endpunkt aufrufen, um Vorhersagen für die Daten ohne Bezeichnung zu generieren.

Zunächst legen Sie die Eingabe fest, indem Sie auf die registrierte Datenressource verweisen. Anschließend rufen Sie den Endpunkt auf, der einen Pipelineauftrag übermittelt. Sie können die URL des Auftrags verwenden, um ihn in Studio zu überwachen. Der Auftrag enthält einen untergeordneten Auftrag, der die Ausführung des (generierten) Bewertungsskripts zum Abrufen der Vorhersagen darstellt.

## Abrufen der Ergebnisse

Wenn der Pipelineauftrag, der den Batchendpunkt aufruft, abgeschlossen ist, können Sie die Ergebnisse anzeigen. Alle Vorhersagen werden in der Datei `predictions.csv` gesammelt, die im Standarddatenspeicher gespeichert ist. Sie können die Datei herunterladen und die Daten visualisieren, indem Sie die folgenden Zellen ausführen. 

In [ ]:
from azure.ai.ml.entities import BatchEndpoint

# create a batch endpoint
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A batch endpoint for classifying diabetes in patients",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the endpoint is created before continuing! A green notification should appear in the studio. </p>

## Create the deployment

A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `BatchDeployment` class. 

Since you're deploying an MLflow model, you don't need a scoring script or define the environment. Azure Machine Learning will automatically create those assets for you. The `MLmodel` file in the `model` folder is used to understand what the expected inputs and outputs are of the model.

You'll deploy a model with the following parameters:

- `name`: Name of the deployment.
- `description`: Optional description to further clarify what the deployment represents.
- `endpoint_name`: Name of the previously created endpoint the model should be deployed to.
- `model`: Name of the registered model.
- `compute`: Compute to be used when invoking the deployed model to generate predictions.
- `instance_count`: Count of compute nodes to use for generating predictions.
- `max_concurrency_per_instance`: Maximum number of parallel scoring script runs per compute node.
- `mini_batch_size`: Number of files passed per scoring script run.
- `output_action`: Each new prediction will be appended as a new row to the output file.
- `output_file_name`: File to which predictions will be appended.
- `retry_settings`: Settings for a mini-batch fails.
- `logging_level`: The log verbosity level. Allowed values are `warning`, `info`, and `debug`. 

Running the following cell will configure and create the deployment.

In [ ]:
from azure.ai.ml.entities import BatchDeployment, BatchRetrySettings
from azure.ai.ml.constants import BatchDeploymentOutputAction

deployment = BatchDeployment(
    name="classifier-diabetes-mlflow",
    description="A diabetes classifier",
    endpoint_name=endpoint.name,
    model=model,
    compute="aml-cluster",
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=2,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)
ml_client.batch_deployments.begin_create_or_update(deployment)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the deployment is completed before continuing! A green notification should appear in the studio. </p>

You can deploy multiple models to an endpoint. You can set the default deployment to specify which model should be used by default when calling a batch endpoint.

In [ ]:
endpoint.defaults = {}

endpoint.defaults["deployment_name"] = deployment.name

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the default deployment is set before continuing! A green notification should appear in the studio. </p>

## Prepare the data for batch predictions

In the `data` folder you'll find CSV files with unlabeled data. You'll create a data asset that points to the files in the `data` folder, which you'll use as input for the batch job.

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

data_path = "./data"
dataset_name = "patient-data-unlabeled"

patient_dataset_unlabeled = Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    description="An unlabeled dataset for diabetes classification",
    name=dataset_name,
)
ml_client.data.create_or_update(patient_dataset_unlabeled)

In [ ]:
patient_dataset_unlabeled = ml_client.data.get(
    name="patient-data-unlabeled", label="latest"
)

## Submit the job

Now that you have deployed a model to a batch endpoint, and have an unlabeled data asset, you're ready to invoke the endpoint to generate predictions on the unlabeled data.

First, you'll define the input by referring to the registered data asset. Then, you'll invoke the endpoint, which will submit a pipeline job. You can use the job URL to monitor it in the Studio. The job will contain a child job that represents the running of the (generated) scoring script to get the predictions.

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

input = Input(type=AssetTypes.URI_FOLDER, path=patient_dataset_unlabeled.id)

In [ ]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name, 
    input=input)

ml_client.jobs.get(job.name)

## Get the results

When the pipeline job that invokes the batch endpoint is completed, you can view the results. All predictions are collected in the `predictions.csv` file that is stored in the default datastore. You can download the file and visualize the data by running the following cells. 

In [ ]:
ml_client.jobs.download(name=job.name, download_path=".", output_name="score")

In [ ]:
with open("predictions.csv", "r") as f:
    data = f.read()

In [ ]:
from ast import literal_eval
import pandas as pd

score = pd.DataFrame(
    literal_eval(data.replace("\n", ",")), columns=["file", "prediction"]
)
score